## Example 1:

In [1]:
import pandas as pd
import math

In [4]:
dat = pd.read_csv('./dataset/DecisionTree', sep='\t')
dat

,Gender,Ownership,Travel Cost,Income,Mode
0,Male,0,Cheap,Low,Bus
1,Male,1,Cheap,Medium,Bus
2,Female,0,Cheap,Low,Bus
3,Male,1,Cheap,Medium,Bus
4,Female,1,Expensive,High,Car
5,Male,2,Expensive,Medium,Car
6,Female,2,Expensive,High,Car
7,Female,1,Cheap,Medium,Train
8,Male,0,Standard,Medium,Train
9,Female,1,Standard,Medium,Train


## Caculation for Entropy of special column - Mode

In [5]:
# Dem so truong hop
items = pd.unique(dat['Mode'])
n_total = dat['Mode'].count()
p = dict()
for item in items:
    n_i = dat['Mode'].loc[dat['Mode'] == item].count()
    p[item] = n_i/n_total

print(p)

{'Bus': 0.4, 'Car': 0.3, 'Train': 0.3}


In [6]:
# Entropy(S)
entropy_s = 0
for item in p.keys():
    pi = p[item]
    ent_i = -pi*math.log2(pi)
    entropy_s += ent_i
print(entropy_s)

1.5709505944546684


### Caculation for Entropy of Gender with Male value

In [7]:
# Lua ra toan bo male
gender = pd.unique(dat['Gender'])
males = dat.loc[dat['Gender'] == gender[0]]
males

,Gender,Ownership,Travel Cost,Income,Mode
0,Male,0,Cheap,Low,Bus
1,Male,1,Cheap,Medium,Bus
3,Male,1,Cheap,Medium,Bus
5,Male,2,Expensive,Medium,Car
8,Male,0,Standard,Medium,Train


In [8]:
# Dem so truong hop
items_male = pd.unique(males['Mode'])
n_total_male = males['Mode'].count()
p_male = dict()
for item in items_male:
    n_i = males['Mode'].loc[males['Mode'] == item].count()
    p_male[item] = n_i/n_total_male
print(p_male)
# Entropy(S)
entropy_male = 0
for item in p_male.keys():
    pi = p_male[item]
    ent_i = -pi*math.log2(pi)
    entropy_male += ent_i
print(f'Entropy(gender=\'Male\') = {entropy_male:.3}')

{'Bus': 0.6, 'Car': 0.2, 'Train': 0.2}
Entropy(gender='Male') = 1.37


### Caculation for Entropy of Gender with Female value

In [9]:
# Lua ra toan bo male
gender = pd.unique(dat['Gender'])
females = dat.loc[dat['Gender'] == gender[1]]
females

,Gender,Ownership,Travel Cost,Income,Mode
2,Female,0,Cheap,Low,Bus
4,Female,1,Expensive,High,Car
6,Female,2,Expensive,High,Car
7,Female,1,Cheap,Medium,Train
9,Female,1,Standard,Medium,Train


In [10]:
# Dem so truong hop
items_female = pd.unique(females['Mode'])
n_total_female = females['Mode'].count()
p_female = dict()
for item in items_female:
    n_i = females['Mode'].loc[females['Mode'] == item].count()
    p_female[item] = n_i/n_total_female
print(p_female)
# Entropy(S)
entropy_female = 0
for item in p_female.keys():
    pi = p_female[item]
    ent_i = -pi*math.log2(pi)
    entropy_female += ent_i
print(f'Entropy(gender=\'Female\') = {entropy_female:.3}')

{'Bus': 0.2, 'Car': 0.4, 'Train': 0.4}
Entropy(gender='Female') = 1.52


### Entropy(Gender)

In [11]:
n_gender = dat['Gender'].count()
n_male = dat['Gender'].loc[dat['Gender'] == 'Male'].count()
p_male = n_male/n_gender
p_female = 1-p_male
entropy_gender = p_male*entropy_male + p_female*entropy_female
print(entropy_gender)

1.4464393446710155


### Gain(S,Gender)

In [12]:
## Gain(S,Gender)
gain = entropy_s - entropy_gender
print(f'Gain(S,Gender) = {gain:.3}')

Gain(S,Gender) = 0.125


## Build *get_gain()* function


In [17]:
from source.DecisionTree import get_gain, get_probability, get_entropy, build_decision_tree, get_fmax

In [18]:
a = get_gain(dat,'Mode','Gender')
print(f'Gain(S,Gender) = {a:.3}')

Gain(S,Gender) = 0.125


In [19]:
# Get all the names of the features
label = 'Mode'
features = dat.drop(label, axis=1).keys()
print(features)

Index(['Gender', 'Ownership', 'Travel Cost', 'Income'], dtype='object')


In [20]:
fmax = get_fmax(dat, label)
print(f'The most meaning feature is "{fmax}"')

The most meaning feature is "Travel Cost"


In [21]:
# After find the gain max
dummydat = dat[[fmax, label]]
dummydat

,Travel Cost,Mode
0,Cheap,Bus
1,Cheap,Bus
2,Cheap,Bus
3,Cheap,Bus
4,Expensive,Car
5,Expensive,Car
6,Expensive,Car
7,Cheap,Train
8,Standard,Train
9,Standard,Train


### Get all unique value

In [22]:
subvalues = pd.unique(dat[fmax])
# All items searching
for item in subvalues:
    print(f'The sub-table with {fmax} = {item}')
    subdummy = dat.loc[dat[fmax] == item]
    print(subdummy)
    entropy_sub = get_entropy(subdummy, label)
    print(f'Entropy = {entropy_sub}\n')

The sub-table with Travel Cost = Cheap
   Gender  Ownership Travel Cost  Income   Mode
0    Male          0       Cheap     Low    Bus
1    Male          1       Cheap  Medium    Bus
2  Female          0       Cheap     Low    Bus
3    Male          1       Cheap  Medium    Bus
7  Female          1       Cheap  Medium  Train
Entropy = 0.7219280948873623

The sub-table with Travel Cost = Expensive
   Gender  Ownership Travel Cost  Income Mode
4  Female          1   Expensive    High  Car
5    Male          2   Expensive  Medium  Car
6  Female          2   Expensive    High  Car
Entropy = 0.0

The sub-table with Travel Cost = Standard
   Gender  Ownership Travel Cost  Income   Mode
8    Male          0    Standard  Medium  Train
9  Female          1    Standard  Medium  Train
Entropy = 0.0



Where, **Entropy = 0** that's meaning the feature is purity and the information can answer what type of it.  

**For example:**  
Entropy(Travel Cost = *Expensive*) = 0 => if *TravelCost = Expensive* then *Mode = Car*.

#### Some others information

In [23]:
features = ['Gender', 'Ownership', 'Travel Cost', 'Income', 'Mode']
# Entropy
for feature in features:
    entropy = get_entropy(dat, feature)
    print(f'Entropy({feature}) = {entropy:.03}')
# Probality
for feature in features:
    p = get_probability(dat,feature)
    print(feature,'=',p,)

Entropy(Gender) = 1.0
Entropy(Ownership) = 1.49
Entropy(Travel Cost) = 1.49
Entropy(Income) = 1.37
Entropy(Mode) = 1.57
Gender = {'Male': 0.5, 'Female': 0.5}
Ownership = {0: 0.3, 1: 0.5, 2: 0.2}
Travel Cost = {'Cheap': 0.5, 'Expensive': 0.3, 'Standard': 0.2}
Income = {'Low': 0.2, 'Medium': 0.6, 'High': 0.2}
Mode = {'Bus': 0.4, 'Car': 0.3, 'Train': 0.3}


### Result

In [24]:
build_decision_tree(dat, label)

{'Travel Cost': {'Cheap': {'Gender': {'Male': 'Bus',
    'Female': {'Ownership': {0: 'Bus', 1: 'Train'}}}},
  'Expensive': 'Car',
  'Standard': 'Train'}}

## Example 2:

In [25]:
dat2 = pd.read_csv('./dataset/DecisionTree2', sep='\t')
dat2.head()

,age,income,student,credit-rating,buys-computer
0,24,high,no,fair,no
1,25,high,no,excellent,no
2,32,high,no,fair,yes
3,43,medium,no,fair,yes
4,44,low,yes,fair,yes


In [26]:
dat2.loc[(dat2.age < 30),  'age.group'] = 'low'
dat2.loc[((dat2.age < 40) & (dat2.age >= 30)),  'age.group'] = 'medium'
dat2.loc[(dat2.age >= 40),  'age.group'] = 'high'

In [29]:
mydat = dat2.drop('age',axis=1)
mydat

,income,student,credit-rating,buys-computer,age.group
0,high,no,fair,no,low
1,high,no,excellent,no,low
2,high,no,fair,yes,medium
3,medium,no,fair,yes,high
4,low,yes,fair,yes,high
5,low,yes,excellent,no,high
6,low,yes,excellent,yes,medium
7,medium,no,fair,no,low
8,low,yes,fair,yes,low
9,medium,yes,fair,yes,high


In [30]:
# Get all the names of the features
label = 'buys-computer'
features = mydat.drop(label,axis=1).keys()
print(features)
# Gain calculation
gain = dict()
fmax = ''
gmax = -1
for feature in features:
    gain[feature] = get_gain(mydat, label, feature)
    if gmax < gain[feature]: 
        gmax = gain[feature]
        fmax = feature 
    print(f'Gain(S,{feature}) = {gain[feature]:.3}')
print(f'The most meaning feature is "{fmax}"')

Index(['income', 'student', 'credit-rating', 'age.group'], dtype='object')
Gain(S,income) = 0.0292
Gain(S,student) = 0.152
Gain(S,credit-rating) = 0.0481
Gain(S,age.group) = 0.247
The most meaning feature is "age.group"


In [32]:
build_decision_tree(mydat, label )

{'age.group': {'low': {'student': {'no': 'no', 'yes': 'yes'}},
  'medium': 'yes',
  'high': {'credit-rating': {'fair': 'yes', 'excellent': 'no'}}}}